In [2]:
%matplotlib inline
import glob
import os
import pandas as pd
import json
import numpy as np
import scipy.stats as ss
from statsmodels.tsa.stattools import grangercausalitytests, adfuller
import matplotlib.pyplot as plt
import math
import warnings
warnings.simplefilter('ignore')


# Nash Bargaining Problem



In [50]:
def poi_bargain(a, b):
    prop = a.sum() / (a.sum() + b.sum())
    return (prop - 0.5) 
    

## Test Cases

In [51]:
#Case when there are 1 stronger to 1 weaker
p1 = np.asarray([0.8,   0.7,  0.9 ])
p2 = np.asarray([0.2,   0.3,  0.1 ])

poi_bargain(p1, p2)

0.29999999999999993

In [52]:

#Case when there are 2 stronger
p1 = np.asarray([0.8,   0.7,  0.9 ])
p2 = np.asarray([0.6,   0.8,  0.8 ])

poi_bargain(p1, p2)

0.021739130434782594

In [53]:
#Case when there are 2 stronger same rate
p1 = np.asarray([0.8,   0.7,  0.9 ])
p2 = np.asarray([0.8,   0.7,  0.9 ])

poi_bargain(p1, p2)

0.0

In [54]:
#Case when there are 2 weaker
p1 = np.asarray([0.2,   0.3,  0.1 ])
p2 = np.asarray([0.1,   0.2,  0.1 ])

poi_bargain(p1, p2)

0.09999999999999998

In [55]:
#Case when there are 1 weaker to 1 stronger
p1 = np.asarray([0.2,   0.3,  0.1 ])
p2 = np.asarray([0.8,   0.7,  0.9 ])

poi_bargain(p1, p2)

-0.30000000000000004

In [56]:
#Case when there are 2 weaker same rate
p1 = np.asarray([0.2,   0.3,  0.1 ])
p2 = np.asarray([0.2,   0.3,  0.1 ])

poi_bargain(p1, p2)

0.0

In [57]:
#Case when there are average 1 weaker
p1 = np.asarray([0.5,   0.5,  0.5 ])
p2 = np.asarray([0.2,   0.3,  0.1 ])

poi_bargain(p1, p2)

0.2142857142857143

In [58]:
#Case when there are average 1 stronger
p1 = np.asarray([0.5,   0.5,  0.5 ])
p2 = np.asarray([0.8,   0.7,  0.9 ])

poi_bargain(p1, p2)

-0.11538461538461536

In [59]:
#Case when there are 2 in average
p1 = np.asarray([0.5,   0.5,  0.5 ])
p2 = np.asarray([0.5,   0.5,  0.5 ])

poi_bargain(p1, p2)

0.0

In [60]:

#Case when both 0
p1 = np.asarray([0,   0,  0 ])
p2 = np.asarray([0,   0,  0 ])

poi_bargain(p1, p2)

nan

In [61]:
#Case when both 1
p1 = np.asarray([1,   1,  1 ])
p2 = np.asarray([1,   1,  1 ])

poi_bargain(p1, p2)

0.0

In [62]:
#Case when there upper and lower bound
p1 = np.asarray([1,   1,  1 ])
p2 = np.asarray([0,   0,  0 ])

poi_bargain(p1, p2)

0.5

In [63]:
#Case mixed
p1 = np.asarray([0.9,   0.2,  0.4,  0.6,   0.7,  0.1 ])
p2 = np.asarray([0.6,   0.2,  0.5,  0.4,   0.4,  0.3 ])

poi_bargain(p1, p2)

0.047169811320754707

In [22]:
df = pd.read_csv("graphs/power_for_02-07-17_12H.csv")
df['in_power'] = df['in_power'].apply(pd.to_numeric)
df['out_power'] = df['out_power'].apply(pd.to_numeric)

In [23]:
df.head()

,in,in_amenity,in_power,influence,out,out_amenity,out_power
0,4037744278,bar,2.0,1.0,404144840,parking,1.000000
1,4037744278,bar,2.0,1.0,3186093217,restaurant,1.000000
2,404144840,parking,1.0,1.0,3648551975,restaurant,1.000000
3,3186093217,restaurant,1.0,1.0,3975094188,bar,1.000000
4,3648551975,restaurant,1.0,1.0,366700371,cafe,0.499287


In [28]:
df.sort_values(['in_power','power'], ascending=False)

,in,in_amenity,in_power,influence,out,out_amenity,out_power,power
27,3581990322,car_sharing,2.000000,1.000000,808930817,cafe,0.497522,0.800794
0,4037744278,bar,2.000000,1.000000,404144840,parking,1.000000,0.666667
1,4037744278,bar,2.000000,1.000000,3186093217,restaurant,1.000000,0.666667
9,432818432,pharmacy,2.000000,1.000000,4311776869,school,1.000000,0.666667
10,432818432,pharmacy,2.000000,1.000000,1243846565,post_box,1.000000,0.666667
28,3581990322,car_sharing,2.000000,1.000000,5276126259,restaurant,1.000000,0.666667
38,4139840859,restaurant,2.000000,1.000000,4055481596,bar,1.000000,0.666667
39,4139840859,restaurant,2.000000,1.000000,358805065,place_of_worship,1.000000,0.666667
56,3976445299,bank,2.000000,1.000000,983388335,pharmacy,1.000000,0.666667
57,3976445299,bank,2.000000,1.000000,3987038855,cafe,1.000000,0.666667
